In [1]:
import pandas as pd # type: ignore
import numpy  as np # type: ignore
import matplotlib.pyplot as plt # type: ignore

https://archive.ics.uci.edu/dataset/360/air+quality

## First steps

1. data cleaning
2. feature and class identification
3. data transformation
4. handling negative values

In [2]:
aq = pd.read_csv('AirQualityUCI.csv', sep=';')
aq = aq[aq.columns[2:-2]].dropna()
classes = list(filter(lambda x: x[-4:] == '(GT)', aq.columns)) #column that end with (GT) are the target classes
features = list(set(aq.columns) - set(classes)) #the rest are features
aq = aq[features]
#replacing commas with periods and converting to float
aq["T"] = aq["T"].apply(lambda x: float(x.replace(',', '.')))
aq["RH"] = aq["RH"].apply(lambda x: float(x.replace(',', '.')))
aq["AH"] = aq["AH"].apply(lambda x: float(x.replace(',', '.')))
#renaming columns in a more readable way
aq = aq.rename(columns={ 'PT08.S1(CO)': 'CO',
                         'PT08.S5(O3)':'OOO',
                         'PT08.S3(NOx)': 'NOX',
                         'PT08.S2(NMHC)': 'NMHC',
                         'PT08.S4(NO2)': 'NOO',
                         'T': 'Temperature'})
#replacing negative values with NaN
for c in aq.columns:
    aq[c] = aq[c].apply(lambda x: x if x >= 0 else np.nan)

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

def cluster_series(s, quantile=0.1, bandwidth= True):
    snnan = s.dropna().to_numpy().reshape(-1, 1) #handle missing data
    if bandwidth:
        bw = estimate_bandwidth(snnan, quantile=quantile)
        print(bw)
        #MeanShift moves towards areas with higher density using a window (bandwidth)
        ms = MeanShift(bandwidth=bw, bin_seeding=True)#bin_seeding=True is faster
    else:
        ms = MeanShift(bandwidth=None, bin_seeding=True)
    ms.fit(snnan)
    labels = ms.labels_
    labels_unique = np.unique(labels)
    n_clusters = len(labels_unique)
    print("number of estimated clusters : %d" % n_clusters)
    print(labels)
    # a DataFrame is created where each value in the original series is paired with its corresponding cluster label.
    dfs = pd.DataFrame([{ "value": snnan[idx][0], "label": l  }   for idx, l in enumerate(labels)])
    r  = list(dfs.groupby(["label"]).min().value) #the minimum value of each cluster
    r.sort()
    return r # sorted list of the minimum values within each cluster, the cluster centers.

#Classifies an input value into one of the identified clusters and return the index of the cluster to which the value belongs.
def find_cluster(value,clusters):
    for i, c in enumerate(clusters):
        if value < c:
            return i + 1
    return len(clusters) + 1


## Quantile

The quantile value determines what percentage of nearest neighbors is considered when estimating the bandwidth.

q = 0.1 (10%) -> small bw -> many small clusters                   
q = 0.8 (80%) -> large bw -> few larger clusters

In [4]:
quantiles = {
    'NOO': 0.09,
    'CO': 0.09,
    'OOO': 0.09,
    'NOX': 0.09,
    'RH': 0.09,
    'AH': 0.09,
    'NMHC': 0.09,
    'Temperature': 0.09
 }

In [ ]:
#calls cluster_series for each feature of aq and stores the result in a dictionary
clusters = { f : cluster_series(aq[f], q)  for f, q in quantiles.items() }

76.20298075853631
number of estimated clusters : 4
[0 0 0 ... 1 1 1]
45.69625180736292
number of estimated clusters : 7
[3 2 3 ... 1 0 1]
84.13469024580135
number of estimated clusters : 4
[1 0 0 ... 0 0 0]
58.44110777444111
number of estimated clusters : 8
[0 1 1 ... 0 0 0]
3.3357468579690805
number of estimated clusters : 6
[0 1 2 ... 4 4 4]
0.07933050828606383
number of estimated clusters : 6
[1 1 1 ... 1 3 3]
56.074296518740965
number of estimated clusters : 8
[3 0 0 ... 3 0 3]
1.8173980842058364
number of estimated clusters : 5
[0 0 0 ... 2 2 4]


In [6]:
clusters #keys are feature from aq, values are clusters centers for each feature

{'NOO': [551.0, 1446.0, 1901.0, 2394.0],
 'CO': [647.0, 1020.0, 1158.0, 1329.0, 1558.0, 1804.0, 1934.0],
 'OOO': [221.0, 1103.0, 1848.0, 2411.0],
 'NOX': [322.0, 1081.0, 1572.0, 1881.0, 2009.0, 2294.0, 2542.0, 2683.0],
 'RH': [9.2, 41.2, 47.8, 53.4, 60.0, 69.4],
 'AH': [0.1847, 0.6213, 0.8874, 1.1361, 1.3657, 1.4787],
 'NMHC': [383.0, 750.0, 854.0, 993.0, 1385.0, 1776.0, 1920.0, 2214.0],
 'Temperature': [0.0, 10.1, 17.8, 22.6, 28.4]}

In [7]:
aq

,NMHC,RH,NOO,NOX,OOO,CO,AH,Temperature
0,1046.0,48.9,1692.0,1056.0,1268.0,1360.0,0.7578,13.6
1,955.0,47.7,1559.0,1174.0,972.0,1292.0,0.7255,13.3
2,939.0,54.0,1555.0,1140.0,1074.0,1402.0,0.7502,11.9
3,948.0,60.0,1584.0,1092.0,1203.0,1376.0,0.7867,11.0
4,836.0,59.6,1490.0,1205.0,1110.0,1272.0,0.7888,11.2
...,...,...,...,...,...,...,...,...
9352,1101.0,29.3,1374.0,539.0,1729.0,1314.0,0.7568,21.9
9353,1027.0,23.7,1264.0,604.0,1269.0,1163.0,0.7119,24.3
9354,1063.0,18.3,1241.0,603.0,1092.0,1142.0,0.6406,26.9
9355,961.0,13.5,1041.0,702.0,770.0,1003.0,0.5139,28.3


### ONE-HOT ENCODING (for cpo which requires binary values)

In [8]:
# loop over each feature in aq
# if the value is not NaN, find the cluster to which it belongs and one-hot encode it (lambda)
# so  the values of each feature are replaced by their respective cluster index or NaN
# then (pd.get_dummies) for each cluster index, a new column is created with the prefix of the feature name
# and each value is gonna be 0 or 1 if the value belongs to the cluster or not
# finally, all the columns are concatenated into a single DataFrame
aq_encoded = pd.concat([pd.get_dummies(aq[f].apply(lambda x: find_cluster(x, clusters[f]) if not np.isnan(x) else x), prefix=f) for f in aq.columns], axis=1)

In [9]:
aq_encoded

,NMHC_2.0,NMHC_3.0,NMHC_4.0,NMHC_5.0,NMHC_6.0,NMHC_7.0,NMHC_8.0,NMHC_9.0,RH_2.0,RH_3.0,...,AH_3.0,AH_4.0,AH_5.0,AH_6.0,AH_7.0,Temperature_2.0,Temperature_3.0,Temperature_4.0,Temperature_5.0,Temperature_6.0
0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
9353,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
9354,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
9355,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
#replaces the values of the Temperature column with the cluster index to which they belong
aq.Temperature.apply(lambda x: find_cluster(x, clusters["Temperature"]) if not np.isnan(x) else x)

0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
9352    4.0
9353    5.0
9354    5.0
9355    5.0
9356    6.0
Name: Temperature, Length: 9357, dtype: float64

In [13]:
pd.get_dummies(_, prefix="Temperature")

,Temperature_2.0,Temperature_3.0,Temperature_4.0,Temperature_5.0,Temperature_6.0
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
9352,0,0,1,0,0
9353,0,0,0,1,0
9354,0,0,0,1,0
9355,0,0,0,1,0


In [14]:
aq_encoded

,NMHC_2.0,NMHC_3.0,NMHC_4.0,NMHC_5.0,NMHC_6.0,NMHC_7.0,NMHC_8.0,NMHC_9.0,RH_2.0,RH_3.0,...,AH_3.0,AH_4.0,AH_5.0,AH_6.0,AH_7.0,Temperature_2.0,Temperature_3.0,Temperature_4.0,Temperature_5.0,Temperature_6.0
0,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
9353,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
9354,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
9355,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
aq_encoded.columns

Index(['NMHC_2.0', 'NMHC_3.0', 'NMHC_4.0', 'NMHC_5.0', 'NMHC_6.0', 'NMHC_7.0',
       'NMHC_8.0', 'NMHC_9.0', 'RH_2.0', 'RH_3.0', 'RH_4.0', 'RH_5.0',
       'RH_6.0', 'RH_7.0', 'NOO_2.0', 'NOO_3.0', 'NOO_4.0', 'NOO_5.0',
       'NOX_2.0', 'NOX_3.0', 'NOX_4.0', 'NOX_5.0', 'NOX_6.0', 'NOX_7.0',
       'NOX_8.0', 'NOX_9.0', 'OOO_2.0', 'OOO_3.0', 'OOO_4.0', 'OOO_5.0',
       'CO_2.0', 'CO_3.0', 'CO_4.0', 'CO_5.0', 'CO_6.0', 'CO_7.0', 'CO_8.0',
       'AH_2.0', 'AH_3.0', 'AH_4.0', 'AH_5.0', 'AH_6.0', 'AH_7.0',
       'Temperature_2.0', 'Temperature_3.0', 'Temperature_4.0',
       'Temperature_5.0', 'Temperature_6.0'],
      dtype='object')

In [17]:
X = ['NMHC_2.0', 'NMHC_3.0', 'NMHC_4.0', 'NMHC_5.0', 'NMHC_6.0', 'NMHC_7.0',
       'NMHC_8.0', 'NMHC_9.0', 'NOX_2.0', 'NOX_3.0', 'NOX_4.0', 'NOX_5.0',
       'NOX_6.0', 'NOX_7.0', 'NOX_8.0', 'NOX_9.0', 'OOO_2.0', 'OOO_3.0',
       'OOO_4.0', 'OOO_5.0', 'RH_2.0', 'RH_3.0', 'RH_4.0', 'RH_5.0', 'RH_6.0',
       'RH_7.0', 'AH_2.0', 'AH_3.0', 'AH_4.0', 'AH_5.0', 'AH_6.0', 'AH_7.0',
       'Temperature_2.0', 'Temperature_3.0', 'Temperature_4.0',
       'Temperature_5.0', 'Temperature_6.0', 'NOO_2.0', 'NOO_3.0', 'NOO_4.0',
       'NOO_5.0']

In [18]:
Y = "CO_2.0" #target that we want to calculate conditional probability
T =aq_encoded[X + [Y] ] #change order and put CO2.O at the end

In [19]:
T

,NMHC_2.0,NMHC_3.0,NMHC_4.0,NMHC_5.0,NMHC_6.0,NMHC_7.0,NMHC_8.0,NMHC_9.0,NOX_2.0,NOX_3.0,...,Temperature_2.0,Temperature_3.0,Temperature_4.0,Temperature_5.0,Temperature_6.0,NOO_2.0,NOO_3.0,NOO_4.0,NOO_5.0,CO_2.0
0,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
9353,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
9354,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
9355,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1


In [20]:
from functools import reduce

## CPO
The cpo function calculates the conditional probability
P(Y∣X), which is the probability of the target feature (Y = "CO_2.0") given a set of selected features from X. It computes this probability by dividing the support of the intersection of X and Y by the support of X.

In [21]:
def cpo(x):
   if not x:
      return 0
   #support of X
   sup_x  = reduce(lambda s1, s2:  s1 & s2, [T[k] for k in x]).mean() # iteratively applies the bitwise AND operation across the columns for the features in x
   #support of X and Y
   sup_xy = reduce(lambda s1, s2:  s1 & s2, [T[k] for k in x + [Y]]).mean() #the proportion of data points where all the features in x and the target Y are 1
   return sup_xy / sup_x if sup_x >= 0.05 else 0 #conditional probability of Y given X


In [22]:
pd.DataFrame([{"item": x} for x in X])

,item
0,NMHC_2.0
1,NMHC_3.0
2,NMHC_4.0
3,NMHC_5.0
4,NMHC_6.0
5,NMHC_7.0
6,NMHC_8.0
7,NMHC_9.0
8,NOX_2.0
9,NOX_3.0


In [23]:
cpo([X[0], X[5]]) #Condition probability of CO_2.0 given NMHC_2.0 and NMHC_7.0

0

## Generiamo n randomici subsets di X

A prescindere da quanto grande sia n, dopo la creazione del subset numero 2**len(X) - 1 si interrompe tutto   

In [24]:
import random

def unique_random_subsets(X, n):
    unique_subsets = set() #set of unique subsets
    X_list = list(X)

    for _ in range(n):
        # If we have all possible subsets, break
        if len(unique_subsets) == (2 ** len(X) - 1):
            break

        # Generate a random subset
        subset_size = random.randint(1, len(X))
        subset = frozenset(random.sample(X_list, subset_size)) #unmutable set --> frozenset to make it hashable
        unique_subsets.add(subset)

    return [list(subset) for subset in list(unique_subsets)]

## Pseudocode 

The `estimate_shapley` function estimates the Shapley value of a given `item` (feature) in the context of a set `X` (features) using `n_samples` random subsets. The Shapley value is a concept from cooperative game theory used to fairly allocate the value of the coalition to individual players.

### Inputs:
- `X`: A set of items.
- `item`: The specific item for which we want to estimate the Shapley value.
- `n_samples`: The number of random subsets to sample from `X` (excluding the `item`).

### Outputs:
- A float representing the estimated Shapley value of the `item`.

### Steps:

1. **Initialize Variables:**
   - `X_diff` := `X` excluding `item`.
   - `N` := 0 (numerator for the Shapley value calculation).
   - `D` := 0 (denominator for the Shapley value calculation).

2. **Generate Random Subsets:**
   - `subsets` := a list of `n_samples` unique random subsets from `X_diff`.
   - `sorted_subsets` := each subset in `subsets` sorted in ascending order.

3. **Iterate Over Each Subset:**
   - For each `subset` in `sorted_subsets`:
     - Compute `di` := `factorial(len(subset)) * factorial(len(X) - len(subset) - 1)`, a combinatorial weight.

4. **Calculate Contributions:**
   - Convert `subset` and `X_diff` to lists.
   - Compute the contribution of adding `item` to the `subset`:
     - `N += di * (cpo(subset + [item]) - cpo(subset))`.
   - Compute the contribution of excluding `item` from the complement of the `subset`:
     - `N += di * (cpo(list(set(X) - set(subset))) - cpo(list(set(X) - set(subset + [item]))))`.
   - Update the denominator `D`:
     - `D += 2 * di`.

5. **Return the Estimated Shapley Value:**
   - If `D > 0`, return `N / D`.
   - Otherwise, return `0`.


The Shapley value is estimated by computing the average marginal contribution of a feature (item) over all possible subsets of features.      
Here, the code is using Shapley values to estimate the contribution of individual features (item) in the set of features X, so how much a feature influence the model's output

In [ ]:
import math
def estimate_shapley(X, item, n_samples):

    X_diff = set(X) - set([item])
    subsets = unique_random_subsets(X_diff, n_samples)
    sorted_subsets = [sorted(subset) for subset in subsets]

    N, D = 0, 0 #numerator and denominator of the Shapley value

    for subset in sorted_subsets:
        # di = weight of the subset based on its size.
        # This is calculated using factorials to give more weight to smaller subsets (Shapley value theory).
        di = math.factorial(len(subset)) * math.factorial(len(X) - len(subset) - 1)

        subset = list(subset)
        X_diff = list(X_diff)

        #margin contribution when the item is added to the subset
        N += di * (cpo(subset + [item]) - cpo(subset))

        #margin contribution when the item is added to the complement of the subset
        N += di * (cpo(list(set(X) - set(subset)))) - cpo(list((set(X)) - set((subset + [item]))))

        D += 2 * di

    return N / D if D > 0 else 0

estimate_shapley(X, random.choice(X), 100) #shapley value of a random feature in X

0.003327017316372034

## Calculate Shapley value for each feature in X

In [ ]:
import matplotlib.pyplot as plt # type: ignore
from tqdm import tqdm # type: ignore
import seaborn as sns # type: ignore

shapley_values = []
for item in tqdm(X, desc="Calculating Shapley values"):
    es = estimate_shapley(X, item, 100)  # Assuming 100 samples for estimation
    shapley_values.append(es)

# Sort attributes by their Shapley values in descending order and combine in tuples
sorted_values = sorted(zip(X, shapley_values), key=lambda x: x[1], reverse=True)

# Prepare data for plotting
attributes = [item for item, _ in sorted_values]
values = [value for _, value in sorted_values]

plt.figure(figsize=(14, 10))
sns.barplot(x=attributes, y=values, palette='inferno')

plt.xlabel('Attribute')
plt.ylabel('Shapley Value')
plt.title('Estimated Shapley Values')
plt.xticks(rotation=45, ha='right')
plt.show()

Calculating Shapley values: 100%|██████████| 41/41 [00:37<00:00,  1.10it/s]


In [ ]:
for item, value in sorted_values:
    print(f"Item :{item} -> Shapley value: {value:.4f}")

Item :NMHC_2.0 -> Shapley value: 0.3945
Item :NOX_3.0 -> Shapley value: 0.3868
Item :OOO_2.0 -> Shapley value: 0.3158
Item :NOO_2.0 -> Shapley value: 0.2902
Item :AH_2.0 -> Shapley value: 0.2716
Item :NMHC_3.0 -> Shapley value: 0.2668
Item :Temperature_2.0 -> Shapley value: 0.2261
Item :RH_3.0 -> Shapley value: 0.2105
Item :RH_2.0 -> Shapley value: 0.2059
Item :AH_5.0 -> Shapley value: 0.1983
Item :RH_4.0 -> Shapley value: 0.1942
Item :Temperature_3.0 -> Shapley value: 0.1853
Item :AH_4.0 -> Shapley value: 0.1805
Item :Temperature_4.0 -> Shapley value: 0.1799
Item :Temperature_5.0 -> Shapley value: 0.1797
Item :RH_5.0 -> Shapley value: 0.1795
Item :Temperature_6.0 -> Shapley value: 0.1609
Item :AH_3.0 -> Shapley value: 0.1561
Item :NOX_2.0 -> Shapley value: 0.1524
Item :AH_6.0 -> Shapley value: 0.1506
Item :RH_7.0 -> Shapley value: 0.1460
Item :AH_7.0 -> Shapley value: 0.1352
Item :NMHC_4.0 -> Shapley value: 0.1039
Item :NOO_3.0 -> Shapley value: 0.0975
Item :NMHC_5.0 -> Shapley value: